Generic Notebook loading and checking each background shapefile or GeoTIFF file.

Also find land cover per ecoregion

In [ ]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
from matplotlib.colors import ListedColormap, BoundaryNorm
import geopandas as gpd
from shapely.geometry import MultiPolygon
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='spherical-berm-323321')

## Load and view land cover TIFF file

In [ ]:
# Path to your TIFF file
tif_path = '/home/users/clelland/Model/Analysis/land_cover_g1.tif'

with rasterio.open(tif_path) as src:
    print("Raster shape from src: ", src.height, src.width)
    print("Raster count (bands):", src.count)

    # Step 2: Read the first band (this SHOULD return a 2D array)
    data = src.read(1)  # Band 1
    print("Data shape after reading:", data.shape)

    # Step 3: Get metadata
    transform = src.transform
    bounds = src.bounds
    nodata = src.nodata

# Step 4: Check if masking is needed
if nodata is not None:
    data = np.ma.masked_equal(data, nodata)

# Step 5: Confirm shape before plotting
if data.ndim != 2:
    raise ValueError(f"Expected 2D array, got shape {data.shape}")

# Get unique values
unique_values = np.unique(data)

print("Unique classes in the TIFF:")
print(unique_values)

In [ ]:
cmap = ListedColormap(['brown', 'green', 'blue', 'yellow', 'red',
                       'purple', 'orange', 'cyan', 'gray'])

# Define projection for polar map (approximate for NSIDC EASE-Grid North)
proj = ccrs.LambertAzimuthalEqualArea(central_latitude=90, central_longitude=0)

# Plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': proj})
extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

# Plot the data with the custom color map
img = ax.imshow(data, extent=extent, origin='upper', transform=proj,
                cmap=cmap, interpolation='none')

# Add map features
ax.coastlines()
ax.gridlines(draw_labels=False)
plt.title("Land Cover Map (EASE-Grid North Projection)")

# Add a colorbar with labels (optional: map the class numbers to labels)
cbar = plt.colorbar(img, ax=ax, shrink=0.6, label="Land cover class")

plt.tight_layout()
plt.show()

## Load land cover shapefile

In [ ]:
# Load shapefile (replace with the path to your shapefile)
shp_path = '/home/users/clelland/Model/Analysis/TEM Land cover shapefile/TEM Land cover.shp'
gdf = gpd.read_file(shp_path)

# Extract unique 'gridcode' values for land cover types
class_values = sorted(gdf['gridcode'].unique())  # Make sure these are in sorted order
print(class_values)

In [ ]:
cmap = ListedColormap(['brown', 'green', 'blue', 'yellow', 'red',
                       'purple', 'orange', 'cyan', 'gray'])

# Create a BoundaryNorm to map gridcode values to color map
norm = BoundaryNorm(boundaries=class_values, ncolors=cmap.N)

# Define projection for polar map (approximate for NSIDC EASE-Grid North)
proj = ccrs.LambertAzimuthalEqualArea(central_latitude=90, central_longitude=0)

# Plot the shapefile
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': proj})
gdf.plot(ax=ax, column='gridcode', cmap=cmap, legend=True,
         legend_kwds={'label': "Land cover class", 'orientation': "horizontal"},
         norm=norm)

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines(draw_labels=False)

# Adjust the plot layout
plt.title("Land Cover Map from Shapefile (EASE-Grid North Projection)")
plt.tight_layout()
plt.show()

## Load geographical shapefiles

In [ ]:
# Load shapefile (replace with the path to your shapefile)
shp_path = '/home/users/clelland/Model/Analysis/Countries shapefile/world-administrative-boundaries.shp'
gdf = gpd.read_file(shp_path)
#selected_countries = gdf[gdf['name'].isin(['Sweden', 'Finland', 'Norway', 'Iceland', 'Greenland'])]
#selected_countries = gdf[gdf['name'].isin(['Russian Federation'])]
#selected_countries = gdf[gdf['name'].isin(['Canada'])]

# Treat Alaska separately
usa_geom = gdf[gdf['iso3'] == 'USA'].geometry.values[0]
if isinstance(usa_geom, MultiPolygon):
    multipolys = list(usa_geom.geoms)
else:
    multipolys = [usa_geom]
alaska_polys = [poly for poly in multipolys if poly.bounds[1] > 50]
alaska_gdf = gpd.GeoDataFrame(geometry=alaska_polys, crs=gdf.crs)

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(8, 10))
#selected_countries.plot(ax=ax, edgecolor='black', color='lightblue')
alaska_gdf.plot(ax=ax, edgecolor='black', color='lightblue')

# Add country names
#for idx, row in selected_countries.iterrows():
#    centroid = row['geometry'].centroid
#    ax.text(centroid.x, centroid.y, row['name'], ha='center', fontsize=10)

ax.set_title("Alaska")
plt.axis('equal')
plt.show()

## Load ecoregion shapefile

### Dinerstein

#### GEE

In [ ]:
# Load the ecoregions
ecoRegions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')

# Apply filters
biome_filter = ee.Filter.inList('BIOME_NUM', [6, 11])
realm_filter = ee.Filter.inList('REALM', ['Nearctic', 'Palearctic'])
combined_filter = ee.Filter.And(biome_filter, realm_filter)
selected_regions = ecoRegions.filter(combined_filter)
region_list = selected_regions.toList(selected_regions.size())

In [ ]:
# Function to compute area in Mha
def compute_area(feature):
    area_m2 = feature.geometry().area()
    area_mha = area_m2.divide(1e10)
    return feature.set('area_Mha', area_mha)

# Map the area computation over the FeatureCollection
regions_with_area = selected_regions.map(compute_area)

# Export or print results
# Option 1: Print in Python (client side)
region_list = regions_with_area.getInfo()
for f in region_list['features']:
    name = f['properties']['ECO_NAME']
    area = f['properties']['area_Mha']
    print(f"{name}: {area:.8f} Mha")

# Extract data to a list of dictionaries
data = []
for f in region_list['features']:
    props = f['properties']
    data.append({
        'ECO_NAME': props['ECO_NAME'],
        'BIOME_NUM': props['BIOME_NUM'],
        'REALM': props['REALM'],
        'area_Mha': props['area_Mha']
    })

# Convert to DataFrame
df = pd.DataFrame(data)
df

# Save to CSV
#df.to_csv('/home/users/clelland/Model/Analysis/ecoregion_area.csv', index=False)

In [ ]:
df = pd.read_csv('/home/users/clelland/Model/Analysis/ecoregion_area.csv')
df

#### Shapefile

In [ ]:
shp_path = '/home/users/clelland/Model/Analysis/RESOLVE shapefile from GEE/resolve_shapefile_from_gee.shp'
gdf = gpd.read_file(shp_path)
gdf = gdf.to_crs(epsg=6931)
selected_ecoregions = gdf[
    gdf['BIOME_NAME'].isin(['Boreal Forests/Taiga', 'Tundra'])]

# Extract unique ecoregions
class_values = sorted(selected_ecoregions['ECO_NAME'].unique())
#print(class_values)
selected_ecoregions.sort_values(by=['BIOME_NUM', 'ECO_NAME'])

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 10))
selected_ecoregions.plot(ax=ax, edgecolor='black', color='lightblue')

ax.set_title("All ecoregions")
plt.axis('equal')
plt.show()

#### RESOLVE extra shapefile

In [ ]:
#shp_path = '/home/users/clelland/Model/Analysis/RESOLVE extra shapefile/selected_resolve_regions.shp'
shp_path = '/home/users/clelland/Model/Analysis/RESOLVE extra shapefile/kalste_shapefile.shp'
gdf = gpd.read_file(shp_path)
selected_ecoregions = gdf[
    gdf['BIOME_NAME'].isin(['Boreal Forests/Taiga', 'Tundra'])]

# Extract unique ecoregions
class_values = sorted(selected_ecoregions['geometry'].unique())
print(class_values)
selected_ecoregions.sort_values(by=['BIOME_NUM', 'ECO_NAME'])

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10, 3))
selected_ecoregions.plot(ax=ax, edgecolor='black', color='lightblue')

ax.set_title("All ecoregions")
plt.axis('equal')
plt.show()

### Olson

In [ ]:
shp_path = '/home/users/clelland/Model/Analysis/WWF shapefile/wwf_terr_ecos.shp'
gdf = gpd.read_file(shp_path)
selected_ecoregions = gdf[
    gdf['BIOME'].isin([6.0, 11.0]) &
    gdf['REALM'].isin(['PA', 'NA'])]

# Extract unique ecoregions
class_values = sorted(selected_ecoregions['ECO_NAME'].unique())
print(class_values)
selected_ecoregions

## Load and view FRI TIFF files

In [ ]:
# Path to your TIFF file
tif_path = '/home/users/clelland/Model/Analysis/fri_firecci_landcover_1972_2020.tif'

with rasterio.open(tif_path) as src:
    print("Raster shape from src: ", src.height, src.width)
    print("Raster count (bands):", src.count)

    # Step 2: Read the first band (this SHOULD return a 2D array)
    data = src.read(1)  # Band 1
    print("Data shape after reading:", data.shape)

    # Step 3: Get metadata
    transform = src.transform
    bounds = src.bounds
    nodata = src.nodata

# Step 4: Check if masking is needed
if nodata is not None:
    data = np.ma.masked_equal(data, nodata)

# Step 5: Confirm shape before plotting
if data.ndim != 2:
    raise ValueError(f"Expected 2D array, got shape {data.shape}")

# Divide values by 365 (as per instruction)
data_scaled = data / 365.0

# Define class bins and labels
bins = [60/365, 100/365, 200/365, 300/365, 400/365, 500/365, 600/365, np.inf]
class_map = np.digitize(data_scaled, bins)

# Mask the classified array wherever the original data was nodata
class_map_masked = np.ma.masked_where(data.mask, class_map)

In [ ]:
# Define a color map with 7 distinct colors
cmap = ListedColormap(['#ffffcc', '#c2e699', '#78c679', '#31a354',
                       '#006837', '#253494', '#54278f'])
norm = BoundaryNorm(boundaries=range(8), ncolors=cmap.N)

# Define projection
proj = ccrs.LambertAzimuthalEqualArea(central_latitude=90, central_longitude=0)
extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

# Plotting
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': proj})
img = ax.imshow(class_map_masked, extent=extent, origin='upper', transform=proj,
                cmap=cmap, norm=norm, interpolation='none')

ax.coastlines()
ax.gridlines(draw_labels=False)
plt.title("FRI map with grouped land cover classes")

# Add colorbar with labels matching bins
cbar = plt.colorbar(img, ax=ax, shrink=0.6, ticks=range(7))
cbar.ax.set_yticklabels([
    "60–100", "100–200", "200–300", "300–400", "400–500", "500–600", ">600"
])
cbar.set_label("FRI (days)")

plt.tight_layout()
plt.show()

In [ ]:
# Path to your TIFF file
tif_path = '/home/users/clelland/Model/Analysis/fri_firecci_no_landcover_1972_2020.tif'

with rasterio.open(tif_path) as src:
    print("Raster shape from src: ", src.height, src.width)
    print("Raster count (bands):", src.count)

    # Step 2: Read the first band (this SHOULD return a 2D array)
    data = src.read(1)  # Band 1
    print("Data shape after reading:", data.shape)

    # Step 3: Get metadata
    transform = src.transform
    bounds = src.bounds
    nodata = src.nodata

# Step 4: Check if masking is needed
if nodata is not None:
    data = np.ma.masked_equal(data, nodata)

# Step 5: Confirm shape before plotting
if data.ndim != 2:
    raise ValueError(f"Expected 2D array, got shape {data.shape}")

# Divide values by 365 (as per instruction)
data_scaled = data / 365.0

# Define class bins and labels
bins = [60/365, 100/365, 200/365, 300/365, 400/365, 500/365, 600/365, np.inf]
class_map = np.digitize(data_scaled, bins)

# Mask the classified array wherever the original data was nodata
class_map_masked = np.ma.masked_where(data.mask, class_map)

In [ ]:
# Define a color map with 7 distinct colors
cmap = ListedColormap(['#ffffcc', '#c2e699', '#78c679', '#31a354',
                       '#006837', '#253494', '#54278f'])
norm = BoundaryNorm(boundaries=range(8), ncolors=cmap.N)

# Define projection
proj = ccrs.LambertAzimuthalEqualArea(central_latitude=90, central_longitude=0)
extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

# Plotting
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': proj})
img = ax.imshow(class_map_masked, extent=extent, origin='upper', transform=proj,
                cmap=cmap, norm=norm, interpolation='none')

ax.coastlines()
ax.gridlines(draw_labels=False)
plt.title("FRI map without grouped land cover")

# Add colorbar with labels matching bins
cbar = plt.colorbar(img, ax=ax, shrink=0.6, ticks=range(7))
cbar.ax.set_yticklabels([
    "60–100", "100–200", "200–300", "300–400", "400–500", "500–600", ">600"
])
cbar.set_label("FRI (days)")

plt.tight_layout()
plt.show()

## Land cover by ecoregion

### Process

In [ ]:
# Load ecoregions
ecoRegions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')

# Apply filters
biome_filter = ee.Filter.inList('BIOME_NUM', [6, 11])
realm_filter = ee.Filter.inList('REALM', ['Nearctic', 'Palearctic'])
combined_filter = ee.Filter.And(biome_filter, realm_filter)
selected_regions = ecoRegions.filter(combined_filter)
region_list = selected_regions.toList(selected_regions.size())

# Load the land cover image
land_cover = ee.Image('users/andyc97/model_shapefiles/TEM_LandCover_Map_V3_Grouped_V1')

# Get all unique land cover classes you care about
landcover_classes = [110, 120, 130, 140, 150, 160, 170, 180, 190]

# Create an empty list to collect results
results = []

# Loop through each region
for i in range(region_list.size().getInfo()):
    if i == 27:
        continue
    feature = ee.Feature(region_list.get(i))

    # Special naming
    if i == 5:
        eco_name = 'Eastern Canadian Shield taiga'
        short_name = 'eashti'
    elif i == 19:
        eco_name = 'Northeast Siberian taiga'
        short_name = 'nesibta'
    elif i == 45:
        eco_name = 'Kalaallit Nunaat Arctic steppe'
        short_name = 'kalste'
    else:
        eco_name = feature.get('ECO_NAME').getInfo()
        words = eco_name.split()
        short_name = (words[0][:4] + words[1][:3]).lower() if len(words) >= 2 else words[0][:7].lower()

    print(f"Processing region: {eco_name} -> {short_name}")

    # Mask the land cover to the feature
    masked_lc = land_cover.select('remapped').clip(feature)

    # Count the number of pixels for each land cover class
    class_counts = masked_lc.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=feature.geometry(),
        scale=4000,
        maxPixels=1e8
    ).getInfo()

    histogram = class_counts.get('remapped', {}) if class_counts else {}

    total_pixels = sum(histogram.values())
    if total_pixels == 0:
        print(f"No land cover pixels found in {eco_name}. Skipping.")
        continue

    # Calculate percentage for each land cover class
    row = {'region': short_name}
    raw_percentages = []
    for lc_class in landcover_classes:
        try:
            count = histogram.get(str(lc_class), 0)
            raw_percentage = (count / total_pixels) * 100
            raw_percentages.append(raw_percentage)
            row[lc_class] = round(raw_percentage, 2)
        except ZeroDivisionError:
            # In case of division by zero, set percentage to 0
            row[lc_class] = 0
            raw_percentages.append(0)

    # Adjust percentages to sum to 100 if needed
    total_percentage = sum(raw_percentages)
    if total_percentage != 100:
        adjustment_factor = 100 / total_percentage if total_percentage > 0 else 1
        for lc_class, percentage in zip(landcover_classes, raw_percentages):
            row[lc_class] = round(percentage * adjustment_factor, 2)

    results.append(row)

# Turn into a pandas DataFrame
df = pd.DataFrame(results)

df.to_csv('/home/users/clelland/Model/Analysis/ecoregion_landcover_percentages.csv', index=False)
df

### Read in from CSV

In [ ]:
df = pd.read_csv('/home/users/clelland/Model/Analysis/ecoregion_landcover_percentages.csv', index_col='region')
rename_dict = {
    '110': 'Deciduous forests',
    '120': 'Coniferous forests',
    '130': 'Mixed forest',
    '140': 'Shrublands',
    '150': 'Herbaceous',
    '160': 'Barrens',
    '170': 'Wetlands',
    '180': 'Shrub tundra',
    '190': 'Other'
}
df.rename(columns=rename_dict, inplace=True)
df

#### By geographical region

In [ ]:
# North America only
df_plot = df.copy().reset_index()
df_plot = df_plot[df_plot['geo'] == 'NAM']

# Sort rows by 'region' alphabetically
df_plot = df_plot.sort_values('region')

# Define which columns to plot (exclude 'region' and 'geo')
plot_columns = [col for col in df_plot.columns if col not in ['region', 'geo']]

# Plot a stacked bar chart
ax = df_plot.set_index('region')[plot_columns].plot(
    kind='bar',
    stacked=True,
    figsize=(10, 7),
    colormap='tab10'
)

# Add labels and title
ax.set_ylabel('Percentage of Land Cover (%)')
ax.set_title('Land Cover Distribution by Ecoregion (North America)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Land Cover Classes', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
#plt.savefig('/home/users/clelland/Model/Analysis/Land cover plots/landcover_by_eco_NA.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Eurasia only
df_plot = df.copy().reset_index()
df_plot = df_plot[df_plot['geo'] == 'EU']

# Sort rows by 'region' alphabetically
df_plot = df_plot.sort_values('region')

# Define which columns to plot (exclude 'region' and 'geo')
plot_columns = [col for col in df_plot.columns if col not in ['region', 'geo']]

# Plot a stacked bar chart
ax = df_plot.set_index('region')[plot_columns].plot(
    kind='bar',
    stacked=True,
    figsize=(10, 7),
    colormap='tab10'
)

# Add labels and title
ax.set_ylabel('Percentage of Land Cover (%)')
ax.set_title('Land Cover Distribution by Ecoregion (Eurasia)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Land Cover Classes', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
#plt.savefig('/home/users/clelland/Model/Analysis/Land cover plots/landcover_by_eco_EU.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Fennoscandia only
df_plot = df.copy().reset_index()
df_plot = df_plot[df_plot['geo'] == 'FS']

# Sort rows by 'region' alphabetically
df_plot = df_plot.sort_values('region')

# Define which columns to plot (exclude 'region' and 'geo')
plot_columns = [col for col in df_plot.columns if col not in ['region', 'geo']]

# Plot a stacked bar chart
ax = df_plot.set_index('region')[plot_columns].plot(
    kind='bar',
    stacked=True,
    figsize=(10, 7),
    colormap='tab10'
)

# Add labels and title
ax.set_ylabel('Percentage of Land Cover (%)')
ax.set_title('Land Cover Distribution by Ecoregion (Fennoscandia)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Land Cover Classes', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
#plt.savefig('/home/users/clelland/Model/Analysis/Land cover plots/landcover_by_eco_FS.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### By biome

In [ ]:
# Boreal biome only
# Make a copy and reset index to access region as a column
df_plot = df.copy().reset_index()
df_plot = df_plot.sort_index(ascending=False)

# Add a region number column based on original row order (starting from 1)
#df_plot['region_number'] = df_plot.index + 1

# Filter for Boreal biome
df_plot = df_plot[df_plot['biome'] == 'BO']

# Sort by region number to preserve original order
#df_plot = df_plot.sort_values('region_number', ascending=False)

# Define columns to plot (exclude 'region', 'geo', 'biome', and 'region_number')
plot_columns = [col for col in df_plot.columns if col not in ['region', 'geo', 'biome', 'region_number']]

# Plot stacked bar chart with region numbers as x-axis labels
#ax = df_plot.set_index('region_number')[plot_columns].plot(
ax = df_plot.set_index('region')[plot_columns].plot(
    kind='barh',
    stacked=True,
    figsize=(10, 12),
    colormap='tab10'
)

# Label and title
ax.set_xlabel('Percentage of Land Cover (%)')
ax.set_title('Land Cover Distribution by Region Number (Boreal biome)')
plt.xticks(rotation=0, ha='center')
plt.ylabel('Region Name')
plt.legend(title='Land Cover Classes', bbox_to_anchor=(0.5, -0.2), loc='lower center', ncol=3)

# Final layout
plt.tight_layout()
#plt.savefig('/home/users/clelland/Model/Analysis/Land cover plots/landcover_by_eco_boreal_horiz.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Tundra biome only
# Make a copy and reset index to access region as a column
df_plot = df.copy().reset_index()
df_plot = df_plot.sort_index(ascending=False)

# Add a region number column based on original row order (starting from 1)
#df_plot['region_number'] = df_plot.index + 1

# Filter for Boreal biome
df_plot = df_plot[df_plot['biome'] == 'TU']

# Sort by region number to preserve original order
#df_plot = df_plot.sort_values('region_number', ascending=False)

# Define columns to plot (exclude 'region', 'geo', 'biome', and 'region_number')
plot_columns = [col for col in df_plot.columns if col not in ['region', 'geo', 'biome', 'region_number']]

# Plot stacked bar chart with region numbers as x-axis labels
#ax = df_plot.set_index('region_number')[plot_columns].plot(
ax = df_plot.set_index('region')[plot_columns].plot(
    kind='barh',
    stacked=True,
    figsize=(10, 12),
    colormap='tab10'
)

# Label and title
ax.set_xlabel('Percentage of Land Cover (%)')
ax.set_title('Land Cover Distribution by Region Number (Tundra biome)')
plt.xticks(rotation=0, ha='center')
plt.ylabel('Region Name')
plt.legend(title='Land Cover Classes', bbox_to_anchor=(0.5, -0.2), loc='lower center', ncol=3)

# Final layout
plt.tight_layout()
#plt.savefig('/home/users/clelland/Model/Analysis/Land cover plots/landcover_by_eco_tundra_horiz.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()